# Entendimento do Negócio

O poder legistativo é de uma enorme importância para a sociedade sendo essencial para o funcionamento de qualquer regime democrático. Se trata de um elo entre o povo e os representantes, este poder deve garantir que todos os brasileiros possuam direitos, garantias e deveres, além de suas liberdades. O legislativo é o responsável pela produção de leis que orientam a sociedade e sediar debates de interesse nacional.

Há duas casas presentes no poder legislativo Câmara dos Deputados e Senado Federal. Um inicia o processo legislativo e o outro a revisa, ou seja ambas se manifestam na elaboração das leis. Inicialmente para este projeto vamos focar no âmbito da Câmara dos deputados! Esta possui os representantes elegidos pela população que apresentam pautas legislativas que influenciam diretamente no dia a dia. 

Tendo essa influência em mente, temos que a Câmara é uma das instituições mais importantes do nosso país e impactante para a população brasileira, segmentos empresariais e grupos minoritários. Uma correta transparência do que tramita na Câmara tornando fácil aproximar o que é de interesse de cada indivíduo para acompanhamento pode trazer uma participação mais ativa dos agentes interessados para a fiscalização e pressão ao poder legislativo, evitando a criação e gastos de dinheiro público com propostas lesivas para o povo e planeta.

Este projeto se insere em fornecer uma forma de facilitar a análise do que será discutido na agenda da câmara semanalmente de acordo com um tema específico, fomentando o engajamento e acompanhamento. Para isso iremos usar os dados abertos da agenda e dos eventos presentes na semana (podem ser reuniões, audiências ou sessões de plenário) e aplicar técnicas de Processamento de Linguagem Natural e Inteligência Artificial para avaliação dos tópicos e discussões.

A agenda pode ser visualizada no site: https://www.camara.leg.br/agenda

A Câmara dos deputados também fornece uma API para acesso de alguns dados abertos, muitos dados não estão portados para a API ou necessitam de técnicas de Webscrap.

A agenda apresenta eventos que possuem documentos relacionados com o assunto a ser tratado, em momento inicial não queremos aprofundar tanto no funcionamento do legislativo como um todo, devido a quantidade de possibilidades que podem a ser aplicadas. Ou seja, este projeto irá focar em um objetivo:
- Facilitar a recuperação de informação e o acompanhamento semanal.

# Coleta e Entendimento dos Dados

In [ ]:
# Bibliotecas adicionais instaladas no ambiente virtual.

# !pip install pandas requests
# !pip install pdfminer.six
# !pip install beautifulsoup4
# !pip install tqdm

Nosso primeiro objetivo é identificar dados que auxiliem no acompanhamento das informações dos eventos do legislativo, e posteriormente aplicar técnicas para resgatá-los.

Algumas perguntas de guia são:
- Como recuperar algo que seja do interesse de determinado tema?
- Como recuperar algo que seja de interesse de outro órgão ou tipo de interessado?

Com isso podemos facilitar a organização e o processo de informação frente a quantia de eventos na agenda, possibilitando gerar alertas caso algo que esteja presente em pauta que impacte os interessados. Posteriormente é possível analisar o quanto determinada pauta ou assunto interfira e seja benéfico para população (de forma subjetiva usando LLMs).

In [1]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup

from datetime import datetime, timedelta
from urllib.parse import urlparse, parse_qs
from pdfminer.high_level import extract_text

Vamos criar um filtro para resgatar eventos apenas da próxima semana. Vamos checar se o dia da semana é final de semana, de forma a condicionar de pegar a agenda dessa semana ou da próxima. Posteriormente podemos colocar isso de forma aberta para o usuário poder selecionar o período de dias que irá recuperar os dados.

In [2]:
today = datetime.today()
weekday = today.weekday()

if weekday < 5:
    start_date = today

    days_until_saturday = 5 - today.weekday() # Sábado é valor 5 
    end_date = today + timedelta(days=days_until_saturday)

elif weekday == 5:
    start_date = today + timedelta(days=1)
    end_date = today + timedelta(days=7)

else:
    start_date = today
    end_date = today + timedelta(days=6)


print(f"Data de início: {start_date.strftime('%Y-%m-%d')}")
print(f"Data de fim: {end_date.strftime('%Y-%m-%d')}")


Data de início: 2025-07-13
Data de fim: 2025-07-19


Após pesquisa na documentação de dados abertos da Câmara foi encontrado o endpoint da API pública (https://dadosabertos.camara.leg.br/swagger/api.html), onde conseguimos pegar os dados iniciais de eventos a partir do filtro fornecido em requisição. Como podem ser retornados diversos eventos nessa requisição precisamos aplicar uma técnica de paginação.

In [3]:
def get_events(start_date, end_date, itens_per_page=100):
    url = "https://dadosabertos.camara.leg.br/api/v2/eventos"
    headers = {
        "Accept": "application/json"
    }

    page = 1
    events = []
    while True:
        params = {
            "dataInicio": start_date,
            "dataFim": end_date,
            "ordenarPor": "dataHoraInicio",
            "itens": itens_per_page,
            "ordem": "ASC",
            "pagina": page
        }

        response = requests.get(url, params=params, headers=headers)

        if response.status_code != 200:
            print(f"Erro ao requisitar página {page}: {response.status_code}")
            break

        data = response.json()["dados"]

        if not data:
            break

        events.extend(data)
        print(f"Página {page} carregada com {len(data)} eventos.")
        page += 1

    return events

In [4]:
events = get_events(
    start_date=start_date.strftime('%Y-%m-%d'),
    end_date=end_date.strftime('%Y-%m-%d'))

Página 1 carregada com 61 eventos.


Conseguimos então ter uma visualização de como os eventos são resgatados, temos informações primordiais em primeiro momento como a `descricao`. Porém ela ainda é muito pequena e não muito rica em informação para permitir uma análise do que o evento se refere. 

In [5]:
events_df = pd.DataFrame(events)
events_df.head()

,id,uri,dataHoraInicio,dataHoraFim,situacao,descricaoTipo,descricao,localExterno,orgaos,localCamara,urlRegistro
0,75866,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-15T09:00,None,Agendada,Sessão Não Deliberativa Solene,Homenagem ao Dia do Esporte\r\n Homenagem ao D...,None,"[{'id': 180, 'uri': 'https://dadosabertos.cama...","{'nome': 'Plenário da Câmara dos Deputados', '...",None
1,76380,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-18T19:00,None,Convocada,Simpósio,Abertura da 19ª edição do projeto Politea\r\n ...,None,"[{'id': 100950, 'uri': 'https://dadosabertos.c...","{'nome': 'Auditório Nereu Ramos', 'predio': No...",None
2,76582,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-15T14:00,None,Convocada,Audiência Pública,Sustentabilidade na Produção e Comercialização...,None,"[{'id': 2004, 'uri': 'https://dadosabertos.cam...","{'nome': 'Anexo II, Plenário 08', 'predio': No...",None
3,76801,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-17T16:00,None,Convocada,Seminário,Aspectos legais e ambientais relativos à impla...,Câmara Municipal de Saquarema,"[{'id': 6174, 'uri': 'https://dadosabertos.cam...","{'nome': None, 'predio': None, 'sala': None, '...",None
4,76873,https://dadosabertos.camara.leg.br/api/v2/even...,2025-07-16T14:30,None,Convocada,Audiência Pública,"Projeto de Lei nº 761/2025, de autoria do Depu...",None,"[{'id': 2001, 'uri': 'https://dadosabertos.cam...","{'nome': 'A Definir', 'predio': None, 'sala': ...",None


Aqui conseguimos visualizar quais os tipos das colunas existem para os eventos, vemos que todos os eventos não apresentam `dataHoraFim`, uma vez que pesquisamos eventos da próxima semana, o que faz todo o sentido essa coluna vir vazia.

In [6]:
events_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              61 non-null     int64 
 1   uri             61 non-null     object
 2   dataHoraInicio  61 non-null     object
 3   dataHoraFim     0 non-null      object
 4   situacao        61 non-null     object
 5   descricaoTipo   61 non-null     object
 6   descricao       61 non-null     object
 7   localExterno    12 non-null     object
 8   orgaos          61 non-null     object
 9   localCamara     61 non-null     object
 10  urlRegistro     0 non-null      object
dtypes: int64(1), object(10)
memory usage: 5.4+ KB


Temos a maioria das colunas com dados do tipo object e com todos os valores não nulos. Todas as colunas são descritivas, ou seja não temos dados de valores numéricos.

- id: ID do Evento
- uri: URL de informações do evento (redireciona para o outro XML da api com detalhes do evento)
- dataHoraInicio: Data estimada para início do evento
- dataHoraFim: Data de finalização, há todos os valores nulos pois provavelmente será só preenchida para eventos finalizados
- Situacao do Evento: Se está confirmado, cancelado, etc.
- descricaoTipo: A descrição do tipo do evento
- descricao: A descrição do evento, resumo do que será discutido
- localExterno: Local externo do evento
- orgaos: A que comissão ou estrutura o evento de refere
- localCamara: o local dentro da câmara que ele irá ocorrer.
- urlRegistro: todos os campos nulos, não descritivo ao que se refere.

Temos que um evento pode estar associado a várias comissões, elas estão dispostas na coluna órgãos. Pode ser interessante realizar um repasse dos dataframes para SQL (isso não será feito em primeiro momento) de forma que possamos pegar informações estatísticas atreladas também as comissões. Não podemos garantir que sempre terá apenas uma comissão associada. 

In [62]:
event_sample = events_df.iloc[8]
event_sample.orgaos

[{'id': 539785,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/539785',
  'sigla': 'SUBSNE25',
  'nome': 'Subcomissão Permanente para tratar do Sistema Nacional de Educação (SNE)',
  'apelido': 'CE - Sistema Nacional de Educação (SNE)',
  'codTipoOrgao': 25,
  'tipoOrgao': 'Subcomissão',
  'nomePublicacao': 'Subcomissão Permanente para tratar do Sistema Nacional de Educação (CE/SUBSNE25)',
  'nomeResumido': 'Sistema Nacional de Educação (SNE)'}]

Temos aqui um exemplo de comissão ao que o evento de refere.

Vamos analisar a URL disposta nos dados recebidos, em primeiro momento é imaginado que a URL redirecione para o site da Câmara contendo este evento, porém não, ao acessar a URL obtemos um XML com informações adicionais e links possivelmente interessantes, o que já redireciona nosso fluxo de coleta de informações..

In [63]:
event_sample.uri

'https://dadosabertos.camara.leg.br/api/v2/eventos/76927'

Vamos realizar uma requisição para esta URL e pesquisar mais detalhes do evento.

In [64]:
event_detail_url = event_sample.uri

response = requests.get(event_detail_url, headers={"Accept": "application/json"})

if response.status_code == 200:
    event_detail = response.json()
    print(f"Detalhes resgatados: {event_detail}")
else:
    print(f"Erro na requisição de detalhes {response.status_code}")


Detalhes resgatados: {'dados': {'uriDeputados': None, 'uriConvidados': None, 'fases': None, 'id': 76927, 'uri': 'https://dadosabertos.camara.leg.br/api/v2/eventos/76927', 'dataHoraInicio': '2025-07-15T09:30', 'dataHoraFim': None, 'situacao': 'Convocada', 'descricaoTipo': 'Audiência Pública', 'descricao': 'Implicações do Sistema Nacional de Educação no processo decisório federativo e no financiamento da educação\r\n 1) JOÃO BATISTA (Confirmado) -  Presidente do  Instituto IDados; \r\n\r\n2) CLAUDIO DE MOURA CASTRO (a Confirmar) - Pesquisador em Educação e doutor em Economia pela Universidade Vanderbilt; \r\n\r\n3) FRANCISCO HEBERT LIMA VASCONSCELOS(Confirmado) - Professor  da Faculdade Federal do Ceará e ex-Secretário de Educação de Sobral/CE; \r\n\r\n4) NELSON AMARAL (Confirmado) - Professor do Programa de Pós-Graduação em Educação e Assessor Especial do Reitor da Universidade Federal de Goiás;\r\n\r\n5)  RICARDO PAES DE BARROS (a Confirmar) - Coordenador de Cátedra do Instituto Ayrton

Temos nos detalhes um pouco mais de informações, como por exemplo os Requerimentos e a url para a Pauta do documento, também temos link para os convidados. Tanto o Requerimento quanto a Pauta, são arquivos que possuem mais informações e podem ser mais descritivos no objetivo que queremos.

In [65]:
event_detail

{'dados': {'uriDeputados': None,
  'uriConvidados': None,
  'fases': None,
  'id': 76927,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/eventos/76927',
  'dataHoraInicio': '2025-07-15T09:30',
  'dataHoraFim': None,
  'situacao': 'Convocada',
  'descricaoTipo': 'Audiência Pública',
  'descricao': 'Implicações do Sistema Nacional de Educação no processo decisório federativo e no financiamento da educação\r\n 1) JOÃO BATISTA (Confirmado) -  Presidente do  Instituto IDados; \r\n\r\n2) CLAUDIO DE MOURA CASTRO (a Confirmar) - Pesquisador em Educação e doutor em Economia pela Universidade Vanderbilt; \r\n\r\n3) FRANCISCO HEBERT LIMA VASCONSCELOS(Confirmado) - Professor  da Faculdade Federal do Ceará e ex-Secretário de Educação de Sobral/CE; \r\n\r\n4) NELSON AMARAL (Confirmado) - Professor do Programa de Pós-Graduação em Educação e Assessor Especial do Reitor da Universidade Federal de Goiás;\r\n\r\n5)  RICARDO PAES DE BARROS (a Confirmar) - Coordenador de Cátedra do Instituto Ayrton Se

A partir daqui temos alguns caminhos interessantes para explorar, como:

- Requerimentos: Temos o link do requerimento associado ao evento
- Pautas: Podemos ter um sumário diferente ao que o evento se refere, nas pautas também podemos encontrar os convidados e representantes necessários para o evento.

A informação de convidados pelo que é visto ao acessar os documentos é repetida em ambos, porém alguns eventos não possuem requerimentos expostos nos detalhes, apenas pautas. Na realidade foi percebido que precisamos muitas vezes acessar o site da Câmara no evento para que conseguirmos acessar os documentos.

Essa resposta de detalhes está defasada em alguns pontos:
- Temos que `links` leva a si mesmo
- `urlDocumentoPauta` também não é válido e leva a um XML simples que não possui tanta informação. Devido a isso devemos achar outra forma de encontrar a pauta. Pensando nisso adentrei em uma pauta manual pelo site da agenda da Câmara, sendo necessário realizar um Webscrap com a url base, alterando apenas o código da pauta.

In [66]:
event_id = event_detail["dados"]["id"]
event_url = f"https://www.camara.leg.br/evento-legislativo/{event_id}"
event_url

'https://www.camara.leg.br/evento-legislativo/76927'

Verificando o site da câmara para extrair a pauta temos:

<img src=" ..\static\img\image.png" width="500px">


Com isso, torna necessário avaliarmos o HTML da páginas para resgatar essas informações. Podemos utilizar o BeautifulSoup para extrair o link necessário de acesso a Pauta.

In [67]:
# Necessário para webscrap para que o requests aja como um navegador
headers = {
    "User-Agent": "Mozilla/5.0"
}

response_event = requests.get(event_url, headers=headers)

if response_event.status_code == 200:
    soup_event = BeautifulSoup(response_event.text, "html.parser")
    itens = soup_event.find_all(class_="links-adicionais__item")

    cleaned_links = {}
    for item in itens:
        texto = item.get_text(strip=True)
        link = item.find("a")["href"] if item.find("a") else None

        cleaned_links[texto] = link
        
        print(f"Texto: {texto}")
        print(f"Link: {link}")
else:
    print(f"Erro ao acessar a página: {response.status_code}")

Texto: Requerimento
Link: https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2929038
Texto: Documentos da reunião
Link: #documentos-modal
Texto: Pauta
Link: https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2936881


In [68]:
soup_event

<!DOCTYPE html>

<!--[if lt IE 7]>      <html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="pt-br"> <![endif]-->
<!--[if IE 7]>         <html class="no-js lt-ie9 lt-ie8" lang="pt-br"> <![endif]-->
<!--[if IE 8]>         <html class="no-js lt-ie9" lang="pt-br"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="pt-br">
<!--<![endif]-->
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="bxxobfdomarwwsu1tu7b9wv9oy0gw2" name="facebook-domain-verification">
<title>
      Acompanhe — Portal da Câmara dos Deputados - Portal da Câmara dos Deputados
  </title>
<meta content="Acompanhamento ao vivo de Sessões da Câmara dos Deputados" name="description"/>
<meta content="#00a300" name="msapplication-TileColor"/>
<meta content="https://www.camara.leg.br/tema/mstile-144x144.png" name="msapplication-TileImage"/>
<meta content="#009e49" name="theme-color"/>
<m

Aqui temos por exemplo o link também para o requerimento e pauta.

In [69]:
cleaned_links

{'Requerimento': 'https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2929038',
 'Documentos da reunião': '#documentos-modal',
 'Pauta': 'https://www.camara.leg.br/proposicoesWeb/prop_mostrarintegra?codteor=2936881'}

In [70]:
# Necessário para webscrap para que o requests aja como um navegador
headers = {
    "User-Agent": "Mozilla/5.0"
}

response_topics = requests.get(cleaned_links['Pauta'], headers=headers)
if response_topics.status_code != 200:
    raise Exception(f"Erro {response_topics.status_code}")

soup_topics = BeautifulSoup(response_topics.text, "html.parser")


Fizemos então uma requisição direta para o arquivo Pauta presente no site da Câmara.

In [71]:
soup_topics

<!--
   Licensed to the Apache Software Foundation (ASF) under one or more
   contributor license agreements.  See the NOTICE file distributed with
   this work for additional information regarding copyright ownership.
   The ASF licenses this file to You under the Apache License, Version 2.0
   (the "License"); you may not use this file except in compliance with
   the License.  You may obtain a copy of the License at

        http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.
-->
<!-- prolog breaks internet explorer, WICKET-2718 -->
<!-- < ? xml version="1.0" encoding="UTF-8" ? > -->
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/lo

Obtemos que a URL disponível no site é inválida... na realidade ela mostra que é feito um redirecionamento automático para outra URL! Tornando o processo de recuperação deste documento bem difícil. Podemos utilizar esta outra URL novamente como uma URL base, alterando apenas o código que direciona para a pauta. URL que é direcionado: https://www.camara.leg.br/internet/ordemdodia/integras/.

Esta foi encontrada acessando diretamente esse redirecionamento, vendo como ele se comportaria em seguida.

In [72]:
# Resgatar o ID que estava como query parameter na URL antiga
parsed_url = urlparse(cleaned_links['Pauta'])
query_params = parse_qs(parsed_url.query)

codteor = query_params.get("codteor", [None])[0]

true_topics_url = f"https://www.camara.leg.br/internet/ordemdodia/integras/{codteor}.htm"
true_topics_url

'https://www.camara.leg.br/internet/ordemdodia/integras/2936881.htm'

In [74]:
headers = {
    "User-Agent": "Mozilla/5.0"
}

response = requests.get(true_topics_url, headers=headers)

if response.status_code != 200:
    raise Exception(f"Erro {response.status_code}")

soup_true_topics = BeautifulSoup(response.text, "html.parser")


Agora sim conseguimos acesso a Pauta! Cabe ressaltar que esta requisição muitas vezes dá erros 403, tornando o acesso as Pautas mais difícil.

In [75]:
soup_true_topics


<!-- RRE INI COP Inicio-->
<!-- ATA INI COP Inicio-->
<!-- LVT INI COP Inicio-->
<!-- ODC INI DEL-->
<html>
<head>
<title>
Pauta - SUBSNE25 - 15/07/2025 09:30
  </title>
<meta content="text/html;charset=utf-8" http-equiv="Content-Type"/>
<basefont face="Arial"/>
</head>
<body topmargin="0">
<center>
<table width="630">
<tr>
<td>
<center>
<img height="80" src="https://www.camara.gov.br/internet/imagens/comissoes/Brasao.jpg" width="70"/>
</center>
</td>
<td width="560">
<font face="Times" new="" roman="">
<br/>
CÂMARA DOS DEPUTADOS
      </font>
</td>
</tr>
</table>
<!-- ODC FIM DEL-->
<p align="CENTER">
<font face="Arial">
<b>
<!-- ODC INI DEL-->
<font size="4">
COMISSÃO DE EDUCAÇÃO
       <br/>
</font>
57ª Legislatura - 3ª Sessão Legislativa Ordinária
      <br/>
<br/>
<!-- ODC FIM DEL-->
</b>
</font>
</p>
<p align="CENTER">
<font face="Arial">
<font size="4">
<b>
<!-- RRE INI DEL-->
<!-- LVT INI DEL-->
<!-- ODC INI DEL-->
PAUTA DE 
<!-- ODC FIM DEL-->
REUNIÃO EXTRAORDINÁRIA<br/>AUDIÊ

Neste ponto temos que a Pauta possui uma diagramação de HTML muito bagunçada e cheia de divisões! Também não podemos garantir que todas as pautas possuiriam as mesmas áreas e tópicos de texto. Devido a isso decidi resgatar todo o texto como um grande bloco, possível posteriormente pegar "a partir de tal palavra", por hora desta forma para este projeto vamos seguir assim.

In [76]:
# Extrai o texto visível da página, sem tags HTML
topics_text = soup_true_topics.get_text(separator="\n", strip=True)

print(topics_text[:2000])

Pauta - SUBSNE25 - 15/07/2025 09:30
CÂMARA DOS DEPUTADOS
COMISSÃO DE EDUCAÇÃO
57ª Legislatura - 3ª Sessão Legislativa Ordinária
PAUTA DE
REUNIÃO EXTRAORDINÁRIA
AUDIÊNCIA PÚBLICA
DIA 15/07/2025
SUBCOMISSÃO PERMANENTE PARA TRATAR DO SISTEMA NACIONAL DE EDUCAÇÃO (SNE)
LOCAL:
Plenário 10
HORÁRIO:
09h30min
TEMA:
"Implicações do Sistema Nacional de Educação no processo decisório federativo e no financiamento da educação"
1) JOÃO BATISTA (Confirmado) - Presidente do Instituto IDados;
2) CLAUDIO DE MOURA CASTRO (a Confirmar) - Pesquisador em Educação e doutor em Economia pela Universidade Vanderbilt;
3) FRANCISCO HEBERT LIMA VASCONSCELOS(Confirmado) - Professor da Faculdade Federal do Ceará e ex-Secretário de Educação de Sobral/CE;
4) NELSON AMARAL (Confirmado) - Professor do Programa de Pós-Graduação em Educação e Assessor Especial do Reitor da Universidade Federal de Goiás;
5) RICARDO PAES DE BARROS (a Confirmar) - Coordenador de Cátedra do Instituto Ayrton Senna e professor do Institutuo de

---

Agora vamos partir para resgatar os requerimentos, temos que nas informações de detalhes do evento há URIs funcionais para acessá-los, a URL direciona para um XML de detalhes e neste XML temos o caminho para o documento inteiro. Vamos resgatá-lo para salvarmos todo o texto e fazermos as análises textuais. Há a possibilidade de criarmos outro DataFrame com informações apenas dos requerimentos, uma vez que um evento pode ter múltiplos.

In [77]:
event_detail['dados']

{'uriDeputados': None,
 'uriConvidados': None,
 'fases': None,
 'id': 76927,
 'uri': 'https://dadosabertos.camara.leg.br/api/v2/eventos/76927',
 'dataHoraInicio': '2025-07-15T09:30',
 'dataHoraFim': None,
 'situacao': 'Convocada',
 'descricaoTipo': 'Audiência Pública',
 'descricao': 'Implicações do Sistema Nacional de Educação no processo decisório federativo e no financiamento da educação\r\n 1) JOÃO BATISTA (Confirmado) -  Presidente do  Instituto IDados; \r\n\r\n2) CLAUDIO DE MOURA CASTRO (a Confirmar) - Pesquisador em Educação e doutor em Economia pela Universidade Vanderbilt; \r\n\r\n3) FRANCISCO HEBERT LIMA VASCONSCELOS(Confirmado) - Professor  da Faculdade Federal do Ceará e ex-Secretário de Educação de Sobral/CE; \r\n\r\n4) NELSON AMARAL (Confirmado) - Professor do Programa de Pós-Graduação em Educação e Assessor Especial do Reitor da Universidade Federal de Goiás;\r\n\r\n5)  RICARDO PAES DE BARROS (a Confirmar) - Coordenador de Cátedra do Instituto Ayrton Senna e professor do 

In [78]:
motion_url = event_detail['dados']['requerimentos']

In [80]:
motion_url = event_detail['dados']['requerimentos'][0]['uri']

response_motion = requests.get(motion_url, headers={"Accept": "application/json"})

if response_motion.status_code == 200:
    print(response_motion.json())

motion_detail = response_motion.json()

motion_detail

{'dados': {'id': 2521204, 'uri': 'https://dadosabertos.camara.leg.br/api/v2/proposicoes/2521204', 'siglaTipo': 'REQ', 'codTipo': 294, 'numero': 106, 'ano': 2025, 'ementa': 'Requer a realização de audiência pública, no âmbito da Subcomissão sobre o Sistema Nacional de Educação (SubSNE), para debater sobre as implicações do SNE no processo decisório federativo e no financiamento da educação.', 'dataApresentacao': '2025-06-09T17:41', 'uriOrgaoNumerador': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/2009', 'statusProposicao': {'dataHora': '2025-07-02T00:00', 'sequencia': 11, 'siglaOrgao': 'CE', 'uriOrgao': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/2009', 'uriUltimoRelator': None, 'regime': '.', 'descricaoTramitacao': 'Notificações', 'codTipoTramitacao': '1005', 'descricaoSituacao': 'Aguardando Providências Internas', 'codSituacao': 936, 'despacho': 'Tornou-se coautor do Requerimento o Deputado Luiz Fernando Vampiro (MDB-SC), com base na aprovação do Requerimento 123/2025 CE na

{'dados': {'id': 2521204,
  'uri': 'https://dadosabertos.camara.leg.br/api/v2/proposicoes/2521204',
  'siglaTipo': 'REQ',
  'codTipo': 294,
  'numero': 106,
  'ano': 2025,
  'ementa': 'Requer a realização de audiência pública, no âmbito da Subcomissão sobre o Sistema Nacional de Educação (SubSNE), para debater sobre as implicações do SNE no processo decisório federativo e no financiamento da educação.',
  'dataApresentacao': '2025-06-09T17:41',
  'uriOrgaoNumerador': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/2009',
  'statusProposicao': {'dataHora': '2025-07-02T00:00',
   'sequencia': 11,
   'siglaOrgao': 'CE',
   'uriOrgao': 'https://dadosabertos.camara.leg.br/api/v2/orgaos/2009',
   'uriUltimoRelator': None,
   'regime': '.',
   'descricaoTramitacao': 'Notificações',
   'codTipoTramitacao': '1005',
   'descricaoSituacao': 'Aguardando Providências Internas',
   'codSituacao': 936,
   'despacho': 'Tornou-se coautor do Requerimento o Deputado Luiz Fernando Vampiro (MDB-SC), com 

Do requerimento temos de interessante:  Ementa, Autores e URL de inteiro Teor.

Até então temos a estrutura:
- Evento
  - Pode possuir vários requerimentos
    - Um requerimento pode possuir vários autores.
  - Possui uma pauta

Diferente das pautas que estão em HTML os documentos são lidos como PDF, ou seja, precisamos baixá-los e depois convertemos para um texto tratável.

In [81]:
import requests

motions_folder = '../data/motions/'

motion_pdf_url = motion_detail['dados']['urlInteiroTeor']

headers = {
    "User-Agent": "Mozilla/5.0",  # finge ser um navegador
    "Accept": "application/pdf"
}

response_motion_pdf = requests.get(motion_pdf_url, headers=headers)

motion_number = motion_detail['dados']['numero']
motion_year = motion_detail['dados']['ano']

pdf_name = os.path.join(motions_folder, f"{motion_number}_{motion_year}.pdf")

if response_motion_pdf.status_code == 200:
    with open(pdf_name, "wb") as f:
        f.write(response_motion_pdf.content)
    print("PDF baixado com sucesso.")
else:
    print(f"Erro {response_motion_pdf.status_code}")


PDF baixado com sucesso.


Vamos testar extrair o texto de um único documento.

In [82]:
texto = extract_text(pdf_name)

# Exibe os primeiros caracteres
print(texto[:1000])

COMISSÃO DE EDUCAÇÃO

REQUERIMENTO Nº          , DE 2025

(Da Sra. ADRIANA VENTURA)

Requer   a   realização   de   audiência   pública,   no
âmbito   da   Subcomissão   sobre   o   Sistema
Nacional de Educação (SubSNE),  para debater
sobre   as   implicações   do   SNE   no   processo
decisório   federativo   e   no   financiamento   da
educação.

Senhor Presidente

Nos termos do art. 255 do Regimento Interno da Câmara dos

Deputados, requeiro a realização de audiência pública conforme deliberação da

Subcomissão sobre o Sistema Nacional de Educação (SNE), para discutir as

implicações do SNE no processo decisório federativo e no financiamento da

educação.   Especificamente,   busca-se   debater   se   a   criação   do   SNE

comprometerá a autonomia decisória e orçamentária dos entes federativos ao

concentrar competências em instâncias deliberativas nacionais, sem previsão

clara de financiamento adicional e sustentável por parte da União.

Para discutir esse importante tema, consi

Com isso temos uma estrutura de resgate dos dados que desejamos! Para melhor Organizar a criação de dataframes, vamos repassar os códigos aqui desenvolvidos para scripts mais funcionais e organizados. Presentes na pasta `src`.

Porém agora temos:
- Eventos com detalhes gerais e Pautas associadas com textos extraídos.
- Requerimentos com PDFs e textos extraídos.


Após extrairmos as informações com os scripts iremos realizar a etapa de EDA em um notebook separado. Como há um processamento de PDFs e múltiplos arquivos. Iremos primeiro gerar um CSV com esses dados para depois apenas explorarmos.


As etapas aqui discutidas serão repassadas para a pasta `src/data_collect/`, que possuirá os scripts `src/data_collect/events.py`, `src/data_collect/motions.py`, `src/data_collect/topics.py`.

Estes scripts irão conter funções que serão chamadas em nosso pipeline de recuperação de dados principal.